# Dataset Cleaning 

The main objectives of this notebook are: 
- Import dataset 
- Get familiar with the data 
- Determine if there are data quality issues 
- Resolve any data quality issues 

In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
import polars as pl 
from google.cloud import bigquery
import numpy as np 
import sys 
import os 


# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from preprocessing import *

In [91]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [92]:
# Create client object
client = bigquery.Client()

In [93]:
QUERY = ("SELECT `date`, `visitStartTime`, `fullVisitorId`, `geoNetwork`.`subContinent`, `geoNetwork`.`country`,"
        "`geoNetwork`.`city`,`device`.`browser`, `device`.`operatingSystem`,`device`.`deviceCategory`,"
        "`trafficSource`.`source`, `h`.`item`.`transactionId`,`hp`.`v2ProductName`,`hp`.`v2ProductCategory` "
        "FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170731`, UNNEST(hits) AS h,"
        "UNNEST(h.product) AS hp;")

In [94]:
run_query = client.query(QUERY)

In [95]:
df = pl.from_arrow(run_query.to_arrow())

In [69]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [70]:
df.shape

(49512, 13)

In [71]:
df.columns

['date',
 'visitStartTime',
 'fullVisitorId',
 'subContinent',
 'country',
 'city',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'source',
 'transactionId',
 'v2ProductName',
 'v2ProductCategory']

In [72]:
df.sample()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501550646,"""1681975491896946073""","""Northern America""","""United States""","""not available in demo dataset""","""Chrome""","""Windows""","""desktop""","""google""",null,"""Google Men's Vintage Badge Tee…","""Home/Apparel/Men's/Men's-T-Shi…"


In [73]:
df.describe()

statistic,date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""49512""",49512.0,"""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""462""","""49512""","""49512"""
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""49050""","""0""","""0"""
"""mean""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,21502.167394,null,null,null,null,null,null,null,null,null,null,null
"""min""","""20170731""",1.5015e9,"""0002457163364254438""","""Australasia""","""Algeria""","""(not set)""","""Amazon Silk""","""(not set)""","""desktop""","""(direct)""","""ORD201707311786""","""1 oz Hand Sanitizer""","""${escCatTitle}"""
"""25%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""max""","""20170731""",1.5016e9,"""9997362993085245352""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""YaBrowser""","""iOS""","""tablet""","""youtube.com""","""ORD201707312663""","""YouTube Youth Short Sleeve Tee…","""Wearables/Men's T-Shirts/"""


In [74]:
df.select(pl.all().is_null().sum())

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,49050,0,0


In [75]:
df.filter(pl.col('transactionId') != 'null')

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Blackout Cap""","""Headgear"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Women's Vintage Hero Te…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Men's Long & Lean Tee C…","""Apparel"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Google 25 oz Red Stainless Ste…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""


In [76]:
df_cleaned = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
df_cleaned.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,str,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [77]:
df = df.filter(~pl.col('v2ProductCategory').str.contains('origCatName'))

In [87]:
df.write_csv('../data/data.csv')

# Encoding the df

In [78]:
encoded_df = target_encoding(df)
encoded_df.write_csv('../data/encoded_df.csv')
encoded_df

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductCategory
str,f64,f64,f64,f64,f64,f64,i32,f64
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
…,…,…,…,…,…,…,…,…
"""7483600664917507409""",0.013646,0.012644,0.011229,0.016248,0.01104,0.019489,1,0.254777
"""7483600664917507409""",0.013646,0.012644,0.011229,0.016248,0.01104,0.019489,1,0.262
"""7483600664917507409""",0.013646,0.012644,0.011229,0.016248,0.01104,0.019489,1,0.262


In [79]:
# encoded_df = label_encoding(df)
# encoded_df

In [ ]:
user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
    pl.col('country'), pl.col('city'), 
    pl.col('browser'),pl.col('operatingSystem'),pl.col('deviceCategory'),
    pl.col('source'),pl.col('transactionId'),pl.col('v2ProductCategory')
)
user_profiles

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductCategory
str,list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[i32],list[f64]
"""5683129422872914929""","[0.013646, 0.013646, … 0.013646]","[0.005886, 0.005886, … 0.005886]","[0.011229, 0.011229, … 0.011229]","[0.016248, 0.016248, … 0.016248]","[0.01104, 0.01104, … 0.01104]","[0.019489, 0.019489, … 0.019489]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""0996099586204064564""","[0.013646, 0.013646, … 0.013646]","[0.005886, 0.005886, … 0.005886]","[0.003759, 0.003759, … 0.003759]","[0.00545, 0.00545, … 0.00545]","[0.01104, 0.01104, … 0.01104]","[0.003369, 0.003369, … 0.003369]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""3094733780828697908""","[0.0, 0.0, … 0.0]","[0.005886, 0.005886, … 0.005886]","[0.000797, 0.000797, … 0.000797]","[0.016248, 0.016248, … 0.016248]","[0.01104, 0.01104, … 0.01104]","[0.003369, 0.003369, … 0.003369]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""8568947535596419848""","[0.013646, 0.013646, … 0.013646]","[0.001715, 0.001715, … 0.001715]","[0.011229, 0.011229, … 0.011229]","[0.00545, 0.00545, … 0.00545]","[0.01104, 0.01104, … 0.01104]","[0.003369, 0.003369, … 0.003369]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""9975858819833819527""","[0.0, 0.0, … 0.0]","[0.005886, 0.005886, … 0.005886]","[0.011229, 0.011229, … 0.011229]","[0.00545, 0.00545, … 0.00545]","[0.01104, 0.01104, … 0.01104]","[0.003369, 0.003369, … 0.003369]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
…,…,…,…,…,…,…,…,…
"""2981325201816358942""","[0.013646, 0.013646, … 0.013646]","[0.005886, 0.005886, … 0.005886]","[0.011229, 0.011229, … 0.011229]","[0.00545, 0.00545, … 0.00545]","[0.01104, 0.01104, … 0.01104]","[0.003369, 0.003369, … 0.003369]","[0, 0, … 1]","[0.0, 0.0, … 0.25]"
"""563473372515552520""","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.000797, 0.000797, … 0.000797]","[0.016248, 0.016248, … 0.016248]","[0.01104, 0.01104, … 0.01104]","[0.019489, 0.019489, … 0.019489]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""574914352744840877""","[0.0, 0.0, … 0.0]","[0.005886, 0.005886, … 0.005886]","[0.000797, 0.000797, … 0.000797]","[0.000566, 0.000566, … 0.000566]","[0.003792, 0.003792, … 0.003792]","[0.003369, 0.003369, … 0.003369]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"


In [81]:
# rec_stream = pl.DataFrame(recommendation_eval_pipeline("2703137619338184529",user_profiles, encoded_df, df))
# rec_stream.write_csv("../data/recommendations.csv")

In [82]:
purchasing_visitors=encoded_df.filter(pl.col('transactionId') != 0).select(pl.col('fullVisitorId')).unique()
purchasing_visitors.write_csv("../data/purchasing_visitors.csv")

In [83]:
purchasing_visitors=encoded_df.filter(pl.col('transactionId') != 0).select(pl.col('fullVisitorId')).unique()
all_recs = []
for i in purchasing_visitors["fullVisitorId"].to_list():
    prec = recommendation_eval_pipeline(i, user_profiles, encoded_df, df)
    all_recs.append(prec)
np.mean(all_recs)

KeyboardInterrupt: 

In [ ]:
np.savetxt("../data/model_accuracy.csv",all_recs)

In [ ]:
df.write_csv("../data/cleaned_google_analytics.csv")

In [84]:
recommendation_pipeline("2703137619338184529", user_profiles, encoded_df, df)

['Waze', 'Headgear', 'Drinkware', 'Bottles', 'Drinkware']

In [86]:
recommendation_eval_pipeline("2703137619338184529", user_profiles, encoded_df, df)

0.25

### Data Pre-processing Pipeline

In [ ]:
def preprocessing_pipeline(df: pl.DataFrame) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """
    df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
    encoded_df = encoding(df)
    
    user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
        pl.col('country'), pl.col('browser'),
        pl.col('transactionId'), pl.col('v2ProductName')
    )
    user_profiles_no_id = user_profiles.drop('fullVisitorId')
    user_profiles_no_id = user_profiles_no_id.to_numpy()
    active_users = encoded_df.drop('fullVisitorId')
    active_users = active_users.to_numpy()

    similarities = calculate_similarity(user_profiles, user_profiles_no_id, active_users)
    sorted_dict = sorted(similarities.items(), key=lambda x: x[1])

    
    df.write_csv("../data/cleaned_google_analytics.csv")
    return recommendation(df,sorted_dict) 

In [ ]:
# preprocessing_pipeline(df)

# Conclusions 
- 